In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from torchvision.models import ResNet18_Weights, resnet18
from tqdm.auto import tqdm
from torchvision import datasets, transforms

#from src.imagenet_x.evaluate import ImageNetX, get_vanilla_transform, ImageNetXImageFolder, load_annotations
#from src.imagenet_x import FACTORS, plots
import os

import shutil

In [8]:
annotations_path = "../data/annotations/"

In [21]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
imagenet_val_path = '../data/ImageNetVal'
transforms = get_vanilla_transform()
dataset = ImageNetX(imagenet_val_path, which_factor='top', partition='val', transform=transforms, filter_prototypes=False)    

In [34]:
class TopFactor(datasets.ImageFolder):
    
    def __init__(self, folder_path, *args, **kwargs):
        super().__init__(folder_path, *args, **kwargs)

datafolder = TopFactor('/media/juliu/T7/TopFactor/')

In [40]:

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

In [4]:
def get_new_annotations(which_factor = 'top', filter_prototypes=False, partition='val'):
    annotations = load_annotations(which_factor=which_factor, partition=partition, filter_prototypes=filter_prototypes)

    if partition == 'val':
        #rows with errors:
        #17758
        #29800
        annotations = annotations.drop([17758, 29800]).reset_index(drop=True)

    if partition == 'train':
        #rows with errors:
        errors = [1760, 2197, 5087, 6654, 10510, 10960]
        annotations = annotations.drop(errors).reset_index(drop=True)
    
    filenames = pd.DataFrame(annotations['file_name'])
    annotations = annotations.drop(columns=['class', 'justification', 'one_word', 'metaclass'])
    classes = pd.from_dummies(annotations.drop(columns=['file_name']))
    
    annotations = filenames.join(classes)
    annotations.columns = ['file_name', 'class']

    return annotations

In [5]:
dftrain = get_new_annotations(partition='val')

In [6]:
dfval = get_new_annotations(partition='train')

In [84]:
dfval['nclass'] = dfval['file_name'].apply(lambda x: x.split('_')[0])

In [14]:
idx_to_class = {
    0: 'multiple_objects',
    1: 'background',
    2: 'color',
    3: 'brighter',
    4: 'darker',
    5: 'style',
    6: 'larger',
    7: 'smaller',
    8: 'object_blocking',
    9: 'person_blocking',
    10: 'partial_view',
    11: 'pattern',
    12: 'pose',
    13: 'shape',
    14: 'subcategory',
    15: 'texture'
}

class_to_idx = {v: k for k, v in idx_to_class.items()}

In [81]:
[os.makedirs(f'/media/juliu/T7/TopFactor/val/{key}') for key in class_to_idx.keys()]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [33]:

for i,item in df5.iterrows():
    itemclass = item[1]
    itemname = item[0]
    shutil.copy(f'{path}ImageNetVal/val/{itemname}', f'{path}TopFactor/{itemclass}/{itemname}')

In [86]:
dfval

,file_name,class,nclass
0,n03633091_15528.JPEG,background,n03633091
1,n02950826_6852.JPEG,background,n02950826
2,n03404251_11352.JPEG,background,n03404251
3,n03930313_3341.JPEG,pattern,n03930313
4,n02099267_827.JPEG,pose,n02099267
...,...,...,...
11222,n03599486_3557.JPEG,pose,n03599486
11223,n03710637_762.JPEG,pattern,n03710637
11224,n02088466_3728.JPEG,pose,n02088466
11225,n02113023_11057.JPEG,background,n02113023


In [88]:
path = '/media/juliu/T7/'

for i,item in dfval.iterrows():
    itemname = item[0]
    itemclass = item[1]
    itemnclass = item[2]
    shutil.copy(f'{path}ImageNetRaw/train/{itemnclass}/{itemname}', f'{path}TopFactor/val/{itemclass}/{itemname}')

In [89]:
val = TopFactor('/media/juliu/T7/TopFactor/val', transform=transform)

In [90]:
val[0]

(tensor([[[-1.5528, -1.7925, -1.7069,  ..., -0.4397, -0.1314, -0.5424],
          [-1.2617, -1.4672, -1.6042,  ..., -0.2856,  0.0569, -0.5938],
          [-1.0904, -1.0219, -1.1589,  ..., -0.2171,  0.0227, -0.6109],
          ...,
          [-1.5014, -1.4843, -1.4843,  ..., -1.7583, -1.5357, -1.4672],
          [-1.4672, -1.3987, -1.3815,  ..., -1.7754, -1.5870, -1.4843],
          [-1.9124, -1.8268, -1.7754,  ..., -1.8097, -1.5185, -1.5014]],
 
         [[-1.2829, -1.4930, -1.4055,  ..., -0.3025,  0.0476, -0.5476],
          [-0.9678, -1.1779, -1.2304,  ..., -0.1625,  0.1877, -0.5301],
          [-0.7927, -0.7577, -0.8978,  ..., -0.1099,  0.1527, -0.4776],
          ...,
          [-0.7052, -0.7402, -0.7577,  ..., -0.9503, -0.4601, -0.4776],
          [-0.8627, -0.8277, -0.7402,  ..., -0.9328, -0.4951, -0.4951],
          [-1.7031, -1.6506, -1.5455,  ..., -0.8803, -0.5126, -0.5301]],
 
         [[-1.2641, -1.6302, -1.4733,  ..., -0.4798, -0.2010, -0.5844],
          [-1.1421, -1.2816,

In [14]:
root = '/mnt/f/MetalabelIntegration/train'

In [7]:
l1 = []

In [16]:
for path, subdirs, files in os.walk(root):
    for name in files:
        l1.append(os.path.join(path, name))

In [19]:
annotations = pd.read_json("../data/annotations/imagenet_x_val_multi_factor.jsonl", lines=True)

In [20]:
annotations

,file_name,class,multiple_objects,background,color,brighter,darker,style,larger,smaller,object_blocking,person_blocking,partial_view,pattern,pose,shape,subcategory,texture,justification,one_word
0,ILSVRC2012_val_00004487.JPEG,762,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,close up of a pan fried sea bass.,sea bass close up
1,ILSVRC2012_val_00003963.JPEG,292,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,sepia image of tiger,digitally altered
2,ILSVRC2012_val_00041992.JPEG,718,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,the bridge is brown,rare view
3,ILSVRC2012_val_00028056.JPEG,635,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,the magnetic compass is on the bronze container,wood shape
4,ILSVRC2012_val_00016832.JPEG,497,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,A dark image of a church.,dark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48863,ILSVRC2012_val_00004973.JPEG,973,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,color is different,color
48864,ILSVRC2012_val_00038750.JPEG,765,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,front view image of rocking chair made by digi...,window digital art
48865,ILSVRC2012_val_00043497.JPEG,409,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,right angle of analog clock with gold metallic...,right angle
48866,ILSVRC2012_val_00005859.JPEG,441,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,A different design of a beer glass.,pattern background


In [32]:
# [17758, 29800]
annotations.iloc[17758]

file_name           ILSVRC2012_val_00026228.JPEG
class                                        272
multiple_objects                               0
background                                     0
color                                          0
brighter                                       0
darker                                         0
style                                          0
larger                                         0
smaller                                        0
object_blocking                                0
person_blocking                                0
partial_view                                   0
pattern                                        0
pose                                           0
shape                                          0
subcategory                                    0
texture                                        0
justification            left view image of wolf
one_word                               left view
Name: 17758, dtype: 

In [33]:
files_not_in_annotations = []

In [41]:
for f in l1:
    if f.split('/')[-1] not in annotations['file_name'].values:
        os.remove(f)
        print(f'{f} removed')

/mnt/f/MetalabelIntegration/train/n01440764/ILSVRC2012_val_00024327.JPEG removed
/mnt/f/MetalabelIntegration/train/n01484850/ILSVRC2012_val_00039534.JPEG removed
/mnt/f/MetalabelIntegration/train/n01496331/ILSVRC2012_val_00000921.JPEG removed
/mnt/f/MetalabelIntegration/train/n01498041/ILSVRC2012_val_00036157.JPEG removed
/mnt/f/MetalabelIntegration/train/n01514668/ILSVRC2012_val_00004306.JPEG removed
/mnt/f/MetalabelIntegration/train/n01514859/ILSVRC2012_val_00026508.JPEG removed
/mnt/f/MetalabelIntegration/train/n01530575/ILSVRC2012_val_00012814.JPEG removed
/mnt/f/MetalabelIntegration/train/n01530575/ILSVRC2012_val_00030224.JPEG removed
/mnt/f/MetalabelIntegration/train/n01531178/ILSVRC2012_val_00013902.JPEG removed
/mnt/f/MetalabelIntegration/train/n01537544/ILSVRC2012_val_00022076.JPEG removed
/mnt/f/MetalabelIntegration/train/n01537544/ILSVRC2012_val_00027498.JPEG removed
/mnt/f/MetalabelIntegration/train/n01558993/ILSVRC2012_val_00018692.JPEG removed
/mnt/f/MetalabelIntegration/

In [35]:
len(files_not_in_annotations)

1132

In [42]:
annotations_val = pd.read_json("../data/annotations/imagenet_x_train_multi_factor.jsonl", lines=True)

In [43]:
root = '/mnt/f/MetalabelIntegration/val'

In [44]:
l1 = []

In [45]:
for path, subdirs, files in os.walk(root):
    for name in files:
        l1.append(os.path.join(path, name))

In [51]:
for f in l1:
    if f.split('/')[-1] in annotations_val['file_name'].values:
        print(f)

/mnt/f/MetalabelIntegration/val/n02815834/n02815834_10594.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_11090.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_5436.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_12804.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_7096.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_11276.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_8089.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_12960.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_7487.JPEG
/mnt/f/MetalabelIntegration/val/n02815834/n02815834_21332.JPEG
/mnt/f/MetalabelIntegration/val/n04428191/n04428191_684.JPEG
/mnt/f/MetalabelIntegration/val/n04428191/n04428191_3031.JPEG
/mnt/f/MetalabelIntegration/val/n04428191/n04428191_24381.JPEG
/mnt/f/MetalabelIntegration/val/n04428191/n04428191_4412.JPEG
/mnt/f/MetalabelIntegration/val/n04428191/n04428191_4519.JPEG
/mnt/f/MetalabelIntegration/val/n04428191/n04428191_7488.JPEG
/m